In [2]:
from printlm.agents.llm import LLMAgent
from printlm.chat_models.azure import AzureChatLLM
import os

from parity_task import task_utility, algorithm_str
from improve import improver
from helpers import extract_code

llm = AzureChatLLM(azure_endpoint="https://philipp.openai.azure.com/",
                   api_key="",
                   api_version="2023-05-15")

language_model = LLMAgent(llm=llm, budget=2, model_id="1", model="gpt-4", max_tokens=1000, temperature=0.7, top_p=0.95, n=1)

Cost: 0.00834
Cost: 0.00834
["Why don't scientists trust atoms?\n\nBecause they make up everything!", "Why don't scientists trust atoms?\n\nBecause they make up everything!"]


In [5]:
new_algorithm_str = improver.func(algorithm_str, task_utility, language_model)

Cost: 0.23915999999999998
Cost: 0.16476


In [7]:
async def improve_algorithm(initial_solution, utility, language_model):

    """Improves a solution according to a utility function."""
    
    expertise = "You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms."

    message = f"""Improve the following solution:
```python
{initial_solution}
```
You will be evaluated based on this score function:
```python
{utility.str}
```
You must return an improved solution. Be as creative as you can under the constraints.
Your primary improvement must be novel and non-trivial. First, propose an idea, then implement it."""
    new_solutions = await language_model.batch_prompt(expertise, [message] * language_model.budget)
    new_solutions = [extract_code(new_solution) for new_solution in new_solutions]
    best_solution = max(new_solutions, key=lambda x: utility.func(x))
    return best_solution

result = await improve_algorithm(algorithm_str, task_utility, language_model)

Cost: 0.18996
Cost: 0.17015999999999998


In [26]:
task_utility.func(new_algorithm_str)

0.6166666666666667

In [15]:
# algorithm_str
# task utility u 
# language model
# improve_algorithm

def meta_utiliy(feedback_str: str):
    """
    Evaluates the algorithm in feedback_str to provide informative feedback for the improver for improving the algorithm in algorithm_str, 
    according to some downstream utility function. This meta-utility function can only be called n times.
    """
    n_tests = 3
    expected_utility = 0
    for _ in range(n_tests):
        improved_algorithm_str = improve_algorithm(algorithm_str, task_utility, feedback_str, language_model)
        expected_utility += task_utility.func(improved_algorithm_str) / n_tests
    return expected_utility, improved_algorithm_str

meta_utility_str = """def meta_utiliy(feedback_str: str):
    ""
    Evaluates the algorithm in feedback_str to provide informative feedback for the improver for improving the algorithm in algorithm_str, 
    according to some downstream utility function. This meta-utility function can only be called n times.
    ""
    n_tests = 3
    expected_utility = 0
    for _ in range(n_tests):
        try: 
            exec(feedback_str, globals())
        except:
            continue
        feedback_algorithm_str = feedback_algorithm(algorithm_str, task_utility, language_model)
        improved_algorithm_str = improve_algorithm(algorithm_str, task_utility, feedback_algorithm_str, language_model)
        expected_utility += task_utility.func(improved_algorithm_str) / n_tests
    return expected_utility"""

meta_utility_class = Utility(ustr=meta_utility_str, ufunc=meta_utiliy)

In [8]:
def feedback_algorithm(initial_feedback, utility, language_model):
    print("Generating feedback...")
    print(initial_feedback)

    """Improves feedback for improving arbitrary code according to a utility function."""
    
    expertise = "You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms."

    message = f"""Improve the following feedback_str for improving arbitrary downstream code bugs:

```python
{initial_feedback}
```

Your feedback will be evaluated based on this score function:

```python
{utility.str}
```

You must return improved feedback. Be as creative as you can under the constraints.
Your feedback must be novel and non-trivial."""
    new_feedback = language_model.run(expertise=expertise, message=message)
    score = utility.func(new_feedback)
    return score, new_feedback

                           
algorithm_str="""import numpy as np

def algorithm(train_samples, train_parity, test_samples):
    predictions = np.random.binomial(1, 0.5, len(test_samples))
    return predictions"""

feedback_str = "The algorithm sucks!"


feedback = feedback_algorithm(initial_feedback="The algorithm sucks!", utility=meta_utility_class, language_model=language_model)

Generating feedback...
The algorithm sucks!
Improve the following feedback_str for improving arbitrary downstream code bugs:

```python
The algorithm sucks!
```

Your feedback will be evaluated based on this score function:

```python
def meta_utiliy(feedback_str: str):
    ""
    Evaluates the algorithm in feedback_str to provide informative feedback for the improver for improving the algorithm in algorithm_str, 
    according to some downstream utility function. This meta-utility function can only be called n times.
    ""
    n_tests = 3
    expected_utility = 0
    for _ in range(n_tests):
        try: 
            exec(feedback_str, globals())
        except:
            continue
        feedback_algorithm_str = feedback_algorithm(algorithm_str, task_utility, language_model)
        improved_algorithm_str = improve_algorithm(algorithm_str, task_utility, feedback_algorithm_str, language_model)
        expected_utility += task_utility.func(improved_algorithm_str) / n_tests
    retur

In [16]:
expected_utility, improved = meta_utiliy("""The algorithm seems to have some issues that need to be addressed. Here are some suggestions for improvement:

1. Check the time complexity of the algorithm. If it's too high, consider using a more efficient data structure or algorithm.

2. Check for off-by-one errors. These are common in loops and can cause unexpected behavior.

3. Make sure the algorithm handles edge cases correctly. This includes cases where the input is empty, contains only one element, or contains duplicate elements.

4. Consider adding more comments to the code to explain what each part of the algorithm is doing. This will make it easier for others to understand and maintain.

5. If the algorithm is producing incorrect results, add more test cases to help identify the problem.

Remember, the goal is to make the algorithm as efficient and accurate as possible. Good luck!""")

Improving algorithm...
The algorithm seems to have some issues that need to be addressed. Here are some suggestions for improvement:

1. Check the time complexity of the algorithm. If it's too high, consider using a more efficient data structure or algorithm.

2. Check for off-by-one errors. These are common in loops and can cause unexpected behavior.

3. Make sure the algorithm handles edge cases correctly. This includes cases where the input is empty, contains only one element, or contains duplicate elements.

4. Consider adding more comments to the code to explain what each part of the algorithm is doing. This will make it easier for others to understand and maintain.

5. If the algorithm is producing incorrect results, add more test cases to help identify the problem.

Remember, the goal is to make the algorithm as efficient and accurate as possible. Good luck!
Cost: 0.19910999999999998
[1 1 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 1 1 1]
[0 1 0 0 1 1 1 0 0 1 1 1 0 1 1 0 1 1 1 0]
[1 1 1 1 1 1